In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.special import gamma
from itertools import combinations

sns.set_style('white')
np.random.seed(42)

In [ ]:
def pos_to_distance(pos: np.ndarray) -> np.ndarray:
    return np.array([np.linalg.norm(c1 - c2) for c1, c2 in combinations(pos, 2)])

def mb(sigma: float, size: tuple) -> np.ndarray:
    return np.sqrt(np.sum([np.square(np.random.normal(0, sigma, size=size)) for _ in range(3)], axis=0))

def mb_pdf(x: np.ndarray, a: float) -> np.ndarray:
    return np.where(x > 0,
        np.sqrt(2 / np.pi) * np.power(x, 2) * np.exp(- np.power(x, 2) / (2 * np.power(a, 2))) / np.power(a, 3),
        0)

def normal_pdf(x: np.ndarray, sigma: float) -> np.ndarray:
    return np.exp(- np.square(x / sigma) / 2) / (sigma * np.sqrt(2 * np.pi))

In [ ]:
PANEL = sns.color_palette("Blues", 10)[-5:]
ALPHA_COLOR = sns.color_palette("Blues", 10)[2]

In [ ]:
D = 10
sigmas = np.linspace(5, 30, 5)
d_distr = {}
for sigma in sigmas:
    d_distance = []
    for _ in tqdm(range(int(5e5))):
        pos = np.array([[0, 0, 0], [D, 0, 0]])
        pos_ = pos + np.random.normal(0, sigma, size=pos.shape)
        d_distance.append(pos_to_distance(pos_) - pos_to_distance(pos))
    d_distr[sigma] = np.stack(d_distance).reshape(-1)

In [ ]:
def p(x: np.ndarray, sigma: float) -> np.ndarray:
    k = 2 / gamma((3 - 2 * np.exp(- sigma)) / 2)
    return k * np.power(x + D, 2 * np.power(1 - np.exp(- sigma / D), np.sqrt(np.pi) / 4)) * np.exp(- np.square(x) / 4 / np.square(sigma))

In [ ]:
for i in range(5):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    sns.set(context='notebook', style='ticks', font_scale=3)
    linewidth = 8
    for v in d_distr.values():
        sns.kdeplot(data=v, linewidth=linewidth, color=ALPHA_COLOR, legend=False, alpha=0.6)
    sns.kdeplot(data=list(d_distr.values())[i], linewidth=linewidth, color=PANEL[i], legend=False)
    plt.yticks([])
    # plt.legend()
    plt.ylabel(None)
    plt.savefig(f'distr-shift-{i}.pdf')

In [ ]:
normals = [np.random.normal(0, i, size=int(1e5)) for i in np.linspace(5, 30, 5) / np.sqrt(2)]
for i in tqdm(range(5)):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    sns.set(context='notebook', style='ticks', font_scale=3)
    linewidth = 8
    for v in normals:
        sns.kdeplot(data=v, linewidth=linewidth, color=ALPHA_COLOR, legend=False, alpha=0.6)
    sns.kdeplot(data=normals[i], linewidth=linewidth, color=PANEL[i], legend=False)
    plt.yticks([])
    # plt.legend()
    plt.ylabel(None)
    plt.savefig(f'gaussian-{i}.pdf')